In [4]:
import pandas as pd

pd.set_option('display.max_columns', None)

In [5]:
base22 = pd.read_excel('data/raw/base_dados.xlsx', sheet_name='PEDE2022')
base23 = pd.read_excel('data/raw/base_dados.xlsx', sheet_name='PEDE2023')
base24 = pd.read_excel('data/raw/base_dados.xlsx', sheet_name='PEDE2024')

base22.rename(
    inplace=True,
    columns={
        'INDE 22': 'INDE',
        'Defas': 'Defasagem',
    }
)

base23.rename(
    inplace=True,
    columns={
        'INDE 2023': 'INDE',
    }
)

base24.rename(
    inplace=True,
    columns={
        'INDE 2024': 'INDE',
    }
)

In [13]:
# Concat dataframes
df22 = base22[['RA', 'INDE', 'Defasagem']]
df22['Ano'] = 2022

df23 = base23[['RA', 'INDE', 'Defasagem']]
df23['Ano'] = 2023

df24 = base24[['RA', 'INDE', 'Defasagem']]
df24 = df24.loc[df24['INDE'] != 'INCLUIR']
df24['INDE'] = df24['INDE'].astype(float)
df24['Ano'] = 2024

df = pd.concat([df22, df23, df24]).sort_values(['RA', 'Ano'])

# Sorting by RA
df['id_ra'] = df['RA'].str.split('-').apply(lambda x: int(x[1]))
df = df.sort_values(['id_ra', 'Ano'])

# Creating new feature based on "Defasagem" 
df['defasagem_ano_seguinte'] = df.groupby('id_ra')['Defasagem'].shift(-1)
df['defasagem_ano_seguinte_diferenca'] = df['defasagem_ano_seguinte'] - df['Defasagem']
df.dropna(subset=['defasagem_ano_seguinte', 'INDE'], inplace=True)

# defasagem_ano_seguinte_diferenca = 1 significa que o aluno se "recuperou" em 1 ano de defasagem
# defasagem_ano_seguinte_diferenca = -1 significa que o aluno defasou em mais 1 ano

In [14]:
df

,RA,INDE,Defasagem,Ano,id_ra,defasagem_ano_seguinte,defasagem_ano_seguinte_diferenca
0,RA-1,5.783000,-1,2022,1,0.0,1.0
1,RA-2,7.055000,0,2022,2,0.0,0.0
4,RA-5,7.427000,0,2022,5,0.0,0.0
5,RA-6,5.848000,-1,2022,6,0.0,1.0
6,RA-7,6.818000,-1,2022,7,0.0,1.0
...,...,...,...,...,...,...,...
896,RA-1214,8.378667,0,2023,1214,0.0,0.0
908,RA-1216,7.710500,-2,2023,1216,0.0,2.0
923,RA-1220,6.467250,-1,2023,1220,-1.0,0.0
928,RA-1221,8.027467,-1,2023,1221,0.0,1.0
